In [ ]:
#Import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os

os.chdir("")   #Set working directory

# Read Data

In [ ]:
#Data from sorare and players
eredivisie_games= pd.read_csv("./eredivisie_player_game_info.csv")
eredivisie_players= pd.read_csv("./eredivisie_player_info.csv")
pl_games= pd.read_csv("./pl_player_game_info.csv")
pl_players= pd.read_csv("./pl_player_info.csv")

#Data from Rapid API
home_df= pd.read_csv("./home_df.csv")
away_df= pd.read_csv("./home_df.csv")

#Fixtures data 2024/204 and standing in 2023
games_dates= pd.read_csv("./games_dates.csv")
standing_df= pd.read_csv("./standing_df.csv")


# Calculate elo-ratings

In [ ]:
#Before we caluclate the ELO, we need to give a value to winning (1), draw (0.5) or loss (0)
values = [1, 0.5, 0]

#Define the condition
home_conditions = [
    games_dates['homegoals'] > games_dates['awaygoals'],
    games_dates['homegoals'] == games_dates['awaygoals'],
    games_dates['homegoals'] < games_dates['awaygoals']
]

away_conditions = [
    games_dates['homegoals'] < games_dates['awaygoals'],
    games_dates['homegoals'] == games_dates['awaygoals'],
    games_dates['homegoals'] > games_dates['awaygoals']
]

#Determine the result scores
games_dates['home_result'] = np.select(home_conditions, values)
games_dates['away_result'] = np.select(away_conditions, values)
games_dates

In [ ]:
#Set parameters
#100 for home advantage and 16 for sensitivity 
bonus=100 
sensitivity=16

#calculate rating based on standing
standing_df['elo_value']=1700 - (((standing_df['rank'] - 1) * (1700 - 1300)) / (max(standing_df['rank']) - 1))

# Create df with all teams
teams = pd.unique(games_dates[['hometeam', 'awayteam']].values.ravel())

elo_df = pd.DataFrame({
    'teamname': teams,
    })

#merge standing and team df together and imputate missing values with lowest rating
elo_df = pd.merge(elo_df, standing_df[['teamname', 'elo_value']], how="left", on=["teamname"])
elo_df['elo_value']=elo_df['elo_value'].fillna(min(elo_df['elo_value']))

elo_data = []

#Sort date values (we again assume one game per day per club.
games_dates = games_dates.sort_values(by='Date').copy()

for _, row in games_dates.iterrows():
    home = row['hometeam']
    away = row['awayteam']
    home_goals = row['homegoals']
    away_goals = row['awaygoals']
    home_result = row['home_result'] 
    away_result = row['away_result']  

        # Get current ratings
    elo_home = elo_df.loc[elo_df['teamname'] == home, 'elo_value'].iloc[0]
    elo_away = elo_df.loc[elo_df['teamname'] == away, 'elo_value'].iloc[0]

        # Expected outcomes
    E_home = 1 / (1 + 10 ** ((elo_away - (elo_home + bonus)) / 400))
    E_away = 1 - E_home

    # Goal difference adjustment (optional)
    goal_diff = abs(home_goals - away_goals) #(we take absolute because the result is also minus)
    multiplier = min(4, goal_diff + 0.5)  # min cap 4, max cap 4 

    #update elo rating, 
    #To reward big wins, you can scale the rating change by a goal difference multiplier, add a 0.5 to avoid getting a 0* everything.
    new_elo_home = elo_home + (sensitivity * multiplier * (home_result - E_home))
    new_elo_away = elo_away + (sensitivity * multiplier * (away_result - E_away))

        # Save back
    elo_df.loc[elo_df['teamname'] == home, 'elo_value'] = new_elo_home
    elo_df.loc[elo_df['teamname'] == away, 'elo_value'] = new_elo_away

    # Save data for record
    row_out = row.copy()
    row_out['elo_value_home'] = elo_home
    row_out['elo_value_away'] = elo_away
    elo_data.append(row_out)

#Save the results into a dataframe
elo_scores_data = pd.DataFrame(elo_data)


# Clean and modify the fixture data

In [ ]:

#Keep only columns that we need
away_df_v2 = away_df[['teamname', 'fixture']]
home_df_v2 = home_df[['teamname', 'fixture']]

#Fixture is an object, and int in games df, thus change type
away_df_v2['fixture'] = pd.to_numeric(away_df_v2['fixture'],errors = 'coerce')
home_df_v2['fixture'] = pd.to_numeric(home_df_v2['fixture'],errors = 'coerce')

#Merge date info and elo values
away_df_v2 = pd.merge(away_df_v2, elo_scores_data[['fixture', 'Date', 'elo_value_away', 'awaygoals']], how="left", on=["fixture"])
away_df_v2 = away_df_v2.rename(columns={"awaygoals": "goals"})

#Also for the home teams
home_df_v2 = pd.merge(home_df_v2, elo_scores_data[['fixture', 'Date', 'elo_value_home', 'homegoals']], how="left", on=["fixture"])
home_df_v2 = home_df_v2.rename(columns={"homegoals": "goals"})

#Now we create values that define the opponent scores
away_df_temp = away_df_v2.drop(columns=['Date'])
#Rename to understand that these are the opponent columns
away_df_temp = away_df_temp.add_prefix('opponent_')
#DO not change the fixture name, since it is the merge var
away_df_temp = away_df_temp.rename(columns={"opponent_fixture": "fixture"})
#Merge
New_home_df_v3 = pd.merge(home_df_v2, away_df_temp, how="left", on=["fixture"])
#Some more renames
New_home_df_v3 = New_home_df_v3.rename(columns={"elo_value_home": "elo_value", "opponent_elo_value_away": "opponent_elo_value"})
New_home_df_v3['place']="home"

#Same for the away df
home_df_temp = home_df_v2.drop(columns=['Date'])
home_df_temp = home_df_temp.add_prefix('opponent_')
home_df_temp = home_df_temp.rename(columns={"opponent_fixture": "fixture"})
New_awaydf_v3 = pd.merge(away_df_v2, home_df_temp, how="left", on=["fixture"])
New_awaydf_v3 = New_awaydf_v3.rename(columns={"elo_value_away": "elo_value", "opponent_elo_value_home": "opponent_elo_value"})
New_awaydf_v3['place']="away"

#Concat both dataframes together
total_df = pd.concat([New_home_df_v3, New_awaydf_v3], ignore_index=True)


# Merge the multiple datasets

In [ ]:
#combine player and game info
eredivisie = pd.merge(eredivisie_games, eredivisie_players, how="left", on=["slug"])
pl= pd.merge(pl_games, pl_players, how="left", on=["slug"])
#combine eredivisie and premier league data
sorare_data = pd.concat([pl, eredivisie], ignore_index=True)
#Only players who played more than 60 minutes
sorare_data = sorare_data[(sorare_data["minsPlayed"]>60)]

#make date variables datetime
sorare_data['Date'] =pd.to_datetime(sorare_data['Date']).dt.date
total_df['Date'] =pd.to_datetime(total_df['Date']).dt.date


In [ ]:
#We want to merge two datatables from two sources. Both sources label the teamnames differently, thus rename
sorare_data.loc[sorare_data['teamname'] =='Sparta Rotterdam', 'teamname'] = 'Sparta Rotterdam'
sorare_data.loc[sorare_data['teamname'] =='RKC Waalwijk', 'teamname'] = 'Waalwijk'	 
sorare_data.loc[sorare_data['teamname'] =='FC Utrecht', 'teamname'] = 'Utrecht'
sorare_data.loc[sorare_data['teamname'] =='FC Groningen', 'teamname'] = 'Groningen'
sorare_data.loc[sorare_data['teamname'] =='PEC Zwolle', 'teamname'] = 'PEC Zwolle'
sorare_data.loc[sorare_data['teamname'] =='NAC Breda', 'teamname'] = 'NAC Breda'
sorare_data.loc[sorare_data['teamname'] =='Almere City FC', 'teamname'] = 'Almere City FC'
sorare_data.loc[sorare_data['teamname'] =='sc Heerenveen', 'teamname'] = 'Heerenveen'
sorare_data.loc[sorare_data['teamname'] =='FC Twente', 'teamname'] = 'Twente'
sorare_data.loc[sorare_data['teamname'] =='AZ', 'teamname'] = 'AZ Alkmaar'
sorare_data.loc[sorare_data['teamname'] =='Go Ahead Eagles', 'teamname'] = 'GO Ahead Eagles'
sorare_data.loc[sorare_data['teamname'] =='Heracles Almelo', 'teamname'] = 'Heracles'
sorare_data.loc[sorare_data['teamname'] =='Feyenoord Rotterdam', 'teamname'] = 'Feyenoord'
sorare_data.loc[sorare_data['teamname'] =='PSV Eindhoven', 'teamname'] = 'PSV Eindhoven'
sorare_data.loc[sorare_data['teamname'] =='Fortuna Sittard', 'teamname'] = 'Fortuna Sittard'
sorare_data.loc[sorare_data['teamname'] =='N.E.C. Nijmegen', 'teamname'] = 'NEC Nijmegen'	
sorare_data.loc[sorare_data['teamname'] =='Willem II', 'teamname'] = 'Willem II'
sorare_data.loc[sorare_data['teamname'] =='AFC Ajax', 'teamname'] = 'Ajax'

#also premier league
sorare_data.loc[sorare_data['teamname'] =='Crystal Palace FC', 'teamname'] =  'Crystal Palace' 
sorare_data.loc[sorare_data['teamname'] =='Arsenal FC', 'teamname'] = 	  'Arsenal' 
sorare_data.loc[sorare_data['teamname'] =='Nottingham Forest FC' , 'teamname'] =	       'Nottingham Forest'
sorare_data.loc[sorare_data['teamname'] =='Chelsea FC', 'teamname'] =	       'Chelsea'
sorare_data.loc[sorare_data['teamname'] =='AFC Bournemouth', 'teamname'] = 	  'Bournemouth' 
sorare_data.loc[sorare_data['teamname'] =='Manchester United FC', 'teamname'] = 'Manchester United' 
sorare_data.loc[sorare_data['teamname'] =='Fulham FC', 'teamname'] = 'Fulham'
sorare_data.loc[sorare_data['teamname'] =='Sheffield United FC' , 'teamname'] =	  'Sheffield Utd' 
sorare_data.loc[sorare_data['teamname'] =='Aston Villa FC', 'teamname'] =  'Aston Villa' 
sorare_data.loc[sorare_data['teamname'] =='Newcastle United FC', 'teamname'] =	  'Newcastle' 
sorare_data.loc[sorare_data['teamname'] =='Tottenham Hotspur FC', 'teamname'] =  'Tottenham' 
sorare_data.loc[sorare_data['teamname'] =='Brentford FC', 'teamname'] =  'Brentford' 
sorare_data.loc[sorare_data['teamname'] =='Southampton FC'	, 'teamname'] = 'Southampton'
sorare_data.loc[sorare_data['teamname'] =='Wolverhampton Wanderers FC', 'teamname'] =  'Wolves' 
sorare_data.loc[sorare_data['teamname'] == 'Burnley FC'	, 'teamname'] =	'Burnley'
sorare_data.loc[sorare_data['teamname'] =='West Ham United FC', 'teamname'] = 'West Ham' 
sorare_data.loc[sorare_data['teamname'] =='Brighton & Hove Albion FC', 'teamname'] = 'Brighton'
sorare_data.loc[sorare_data['teamname'] =='Leicester City FC'	, 'teamname'] = 'Leicester'
sorare_data.loc[sorare_data['teamname'] =='Liverpool FC' , 'teamname'] = 'Liverpool' 
sorare_data.loc[sorare_data['teamname'] =='Ipswich Town FC' , 'teamname'] =	'Ipswich'
sorare_data.loc[sorare_data['teamname'] =='Everton FC'	, 'teamname'] =  'Everton' 
sorare_data.loc[sorare_data['teamname'] =='Luton Town FC', 'teamname'] = 'Luton' 
sorare_data.loc[sorare_data['teamname'] =='Manchester City FC', 'teamname'] = 'Manchester City' 

In [ ]:
#Merge scores and games with eachother    
final_file = pd.merge(sorare_data, total_df, how="left", on=["Date", "teamname"], indicator=True)   
final_file['_merge'].value_counts()
#The observations that do not merge are from before 2024 or different competition (for example national teams)

# Create dataframe for goalkeepers

In [ ]:
# Apply condition using loc for price classification
keepers = final_file[final_file['position']=="Goalkeeper"].copy()

#adjust the decesive scores
values = [10, 20, 25, 10]

#Based on their original decisive score and goals conceded, we adjust their decesive score
decisive_conditions = [
    (keepers['decisive'] ==5) & (keepers['opponent_goals']>2),
    (keepers['decisive'] ==15) & (keepers['opponent_goals']>2),
    (keepers['decisive'] ==35)& (keepers['opponent_goals']>2),
    (keepers['decisive'] >59) & (keepers['opponent_goals']>2)
]

#calculate new score
keepers['extra_decisive'] = np.select(decisive_conditions, values)
keepers['new_so5']= keepers['so5score']+keepers['extra_decisive']
keepers = keepers[keepers['_merge'] =="both"]

#Select subset
keepers_v2=keepers[['opponent_elo_value', 'elo_value', 'new_so5', 'so5score', 'teamname', 'opponent_teamname', 'place']]

#create variable with the average elo rating
keepers_v2['avg_elo_value'] = keepers_v2.groupby(['teamname'])['elo_value'].transform('mean')
#Calculate which quintile the team should be in
keepers_v2['quintiles'] = pd.qcut(keepers_v2['avg_elo_value'], 5, labels=False)

#calculate the average score before and after the scoiring matrix change
keepers_v2['AVG_score_new'] = keepers_v2.groupby(['quintiles'])['new_so5'].transform('mean')
keepers_v2['AVG_score'] = keepers_v2.groupby(['quintiles'])['so5score'].transform('mean')

#Do this again but then per opposition quintiles
keepers_v2['avg_elo_value_opp'] = keepers_v2.groupby(['opponent_teamname'])['opponent_elo_value'].transform('mean')
#Calculate which quintile the team should be in
keepers_v2['quintiles_opp'] = pd.qcut(keepers_v2['avg_elo_value_opp'], 5, labels=False)

#calculate the average score before and after the scoiring matrix change
keepers_v2['AVG_score_new_opp'] = keepers_v2.groupby(['quintiles_opp'])['new_so5'].transform('mean')
keepers_v2['AVG_score_opp'] = keepers_v2.groupby(['quintiles_opp'])['so5score'].transform('mean')

#deciles run from 0 to 4, thus add 1 for readability
keepers_v2['quintiles'] = keepers_v2['quintiles']+1
keepers_v2['quintiles_opp'] = keepers_v2['quintiles_opp']+1

#create dataframes with one average value per decile
keepers_def=keepers_v2.drop_duplicates('quintiles')
keepers_def=keepers_def.sort_values('quintiles')

keepers_opp_def=keepers_v2.drop_duplicates('quintiles_opp')
keepers_opp_def=keepers_opp_def.sort_values('quintiles_opp')

# Analyse the data

In [ ]:
#define function the create figures
def figures_gk(df, quintiles, scoring, scoring_new, name):
    # Create side-by-side plots with shared y-axis
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    
    # First barplot
    sns.barplot(data=df, x=quintiles, y=scoring, ax=ax1)
    ax1.set_title('With negative decisives')
    ax1.set_xlabel('Quintiles')
    ax1.set_ylabel('Average score within quintile')
    ax1.set_xticklabels(['Relegation', 'Lower mid-table', 'Mid-table', 'Challengers', 'Elite'])

    # Add value labels to each bar
    for container in ax1.containers:
        ax1.bar_label(container, fmt='%.0f', padding=3)
    
    # Second barplot
    sns.barplot(data=df, x=quintiles, y=scoring_new, ax=ax2)
    ax2.set_title('No negative decisives')
    ax2.set_xlabel('Quintiles')
    ax2.set_ylabel('')  # No label here since y-axis is shared
    ax2.set_xticklabels(['Relegation', 'Lower mid-table', 'Mid-table', 'Challengers', 'Elite'])
    
    # Add value labels to each bar
    for container in ax2.containers:
        ax2.bar_label(container, fmt='%.0f', padding=3)
    
    # Improve layout
    plt.tight_layout()
    plt.savefig(f"score_{name}.png", dpi=300, bbox_inches='tight')


In [ ]:
#print the the plots
figures_gk(keepers_def, "quintiles", "AVG_score", "AVG_score_new", "GK_quintiles_scores")
figures_gk(keepers_opp_def, "quintiles_opp", "AVG_score_opp", "AVG_score_new_opp", "GK_quintiles_scores_opposition")

# Home and away analyses

In [ ]:
keepers_v2_home = keepers_v2[keepers_v2['place']=="home"]
keepers_v2_away = keepers_v2[keepers_v2['place']=="away"]

#calculate the average score before and after the scoiring matrix change
keepers_v2_home['AVG_score_new'] = keepers_v2_home.groupby(['quintiles'])['new_so5'].transform('mean')
keepers_v2_home['AVG_score'] = keepers_v2_home.groupby(['quintiles'])['so5score'].transform('mean')

#calculate the average score before and after the scoiring matrix change
keepers_v2_away['AVG_score_new'] = keepers_v2_away.groupby(['quintiles'])['new_so5'].transform('mean')
keepers_v2_away['AVG_score'] = keepers_v2_away.groupby(['quintiles'])['so5score'].transform('mean')

#create dataframes with one average value per decile
keepers_home_def=keepers_v2_home.drop_duplicates('quintiles')
keepers_home_def=keepers_home_def.sort_values('quintiles')

keepers_away_def=keepers_v2_away.drop_duplicates('quintiles')
keepers_away_def=keepers_away_def.sort_values('quintiles')

In [ ]:
#print the the plots
# Create side-by-side plots with shared y-axis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
    
# First barplot
sns.barplot(data=keepers_home_def, x="quintiles", y="AVG_score_new", ax=ax1)
ax1.set_title('Home')
ax1.set_xlabel('Quintiles')
ax1.set_ylabel('Average score within quintile')
ax1.set_xticklabels(['Relegation', 'Lower mid-table', 'Mid-table', 'Challengers', 'Elite'])

# Add value labels to each bar
for container in ax1.containers:
    ax1.bar_label(container, fmt='%.0f', padding=3)
    
# Second barplot
sns.barplot(data=keepers_away_def, x="quintiles", y="AVG_score_new", ax=ax2)
ax2.set_title('Away')
ax2.set_xlabel('Quintiles')
ax2.set_ylabel('')  # No label here since y-axis is shared
ax2.set_xticklabels(['Relegation', 'Lower mid-table', 'Mid-table', 'Challengers', 'Elite'])
    
# Add value labels to each bar
for container in ax2.containers:
    ax2.bar_label(container, fmt='%.0f', padding=3)
    
    # Improve layout
plt.tight_layout()
plt.savefig(f"Home_and_away_results.png", dpi=300, bbox_inches='tight')
